### Loading text file (Parsed PDF into .txt file)

In [1]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import os

dir_path = './AAPLReports'

Data_Book = []

for filename in os.listdir(dir_path):
    if filename.endswith('.md'):
        file_path = os.path.join(dir_path, filename)
        md_loader = UnstructuredMarkdownLoader(file_path)
        Data_Book.extend(md_loader.load_and_split())

In [2]:
len(Data_Book)

92

In [3]:
Data_Book[:2]

[Document(metadata={'source': './AAPLReports/2023 Q3 AAPL.md'}, page_content='UNITED STATES SECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\nFORM 10-Q\n\n(Mark One)\n\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the quarterly period ended July 1, 2023\n\nor\n\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the transition period from to .\n\nCommission File Number: 001-36743\n\nApple Inc.\n\n(Exact name of Registrant as specified in its charter)\n\nCalifornia 94-2404110 (State or other jurisdiction (I.R.S. Employer Identification No.) of incorporation or organization)\n\nOne Apple Park Way\n\nCupertino, California 95014\n\n(Address of principal executive offices)\n\n(Zip Code)\n\n(408) 996-1010\n\n(Registrant’s telephone number, including area code)\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\nTitle of each class Trading symbol(s) Name of each excha

In [4]:
Data_Book[0].page_content

'UNITED STATES SECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\nFORM 10-Q\n\n(Mark One)\n\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the quarterly period ended July 1, 2023\n\nor\n\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the transition period from to .\n\nCommission File Number: 001-36743\n\nApple Inc.\n\n(Exact name of Registrant as specified in its charter)\n\nCalifornia 94-2404110 (State or other jurisdiction (I.R.S. Employer Identification No.) of incorporation or organization)\n\nOne Apple Park Way\n\nCupertino, California 95014\n\n(Address of principal executive offices)\n\n(Zip Code)\n\n(408) 996-1010\n\n(Registrant’s telephone number, including area code)\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\nTitle of each class Trading symbol(s) Name of each exchange on which registered Common Stock, $0.00001 par value per share AAPL The N

### LLM using Azure OpenAI

In [6]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv() # Load AzureAI and Langsmith credentials

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  
    api_version="2024-08-01-preview", 
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Graph and GraphIndex Creation

In [7]:
import os
import pickle
from tqdm import tqdm
from langchain_experimental.graph_transformers import LLMGraphTransformer

checkpoint_file = 'aapl_processed_docs_checkpoint.pkl'

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'rb') as f:
        processed_indexes = pickle.load(f)
else:
    processed_indexes = []

start_index = len(processed_indexes) # Resume processing docs from start_index

llm_transformer = LLMGraphTransformer(llm=llm)

for i, doc in enumerate(tqdm(Data_Book[start_index:], desc="Processing Documents", initial=start_index, total=len(Data_Book))):
    graph_doc = llm_transformer.convert_to_graph_documents([doc])
    processed_indexes.extend(graph_doc)

    # Save progress every 1000 documents (adjust as needed)
    if len(processed_indexes) % 10 == 0 or len(processed_indexes) == len(Data_Book):
        with open(checkpoint_file, 'wb') as f:
            pickle.dump(processed_indexes, f)
    
graphs_documents = processed_indexes

Processing Documents: 100%|██████████| 92/92 [31:56<00:00, 20.83s/it]


## Drawing and Saving the Graph

In [14]:
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

graph = NetworkxEntityGraph()

for graph_document in graphs_documents:
    for node in graph_document.nodes:
        graph.add_node(node.id)

    for edge in graph_document.relationships:
        graph._graph.add_edge(
                edge.source.id,
                edge.target.id,
                relation=edge.type,
            )

In [9]:
import pickle
pickle.dump(graph, open('AAPL-Sec-10Q-Graph.pickle', 'wb'))

In [10]:
import pickle
graph = pickle.load(open('AAPL-Sec-10Q-Graph.pickle', 'rb'))

### GraphQAChain

In [15]:
from langchain.chains import GraphQAChain

chain = GraphQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True
)

In [16]:
question = """
How has Apple's total net sales changed over time?
"""

chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Apple
Full Context:
Apple FILE_LAWSUIT Epic_Games
Apple OPERATE App_Store
Apple PETITION Circuit_Court

> Finished chain.


{'query': "\nHow has Apple's total net sales changed over time?\n",
 'result': "I don't know."}

In [13]:
question = """
Can any trends be identified in Apple's Services segment revenue over the reported periods?
"""

chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Apple, Services
Full Context:
Apple FILE_LAWSUIT Epic_Games
Apple OPERATE App_Store
Apple PETITION Circuit_Court
Services GROSS_MARGIN_INCREASE Third Quarter Of 2023
Services GROSS_MARGIN_DECREASE Third Quarter Of 2022
Services GROSS_MARGIN_DECREASE 2023
Services GROSS_MARGIN_DECREASE 2022
Services GROSS_MARGIN_INCREASE Higher Services Net Sales
Services GROSS_MARGIN_DECREASE Foreign Currencies
Services GROSS_MARGIN_DECREASE U.S. Dollar
Services GROSS_MARGIN_DECREASE Higher Services Costs
Services GROSS_MARGIN_DECREASE Improved Leverage
Services GROSS_MARGIN_DECREASE First Nine Months Of 2023
Services GROSS_MARGIN_DECREASE Services Costs
Services GROSS_MARGIN_DECREASE Services Gross Margin Percentage
Services GROSS_MARGIN_DECREASE Year-Over-Year Services Gross Margin Percentage
Services IMPACT Net Sales
Services INCLUDES Advertising
Services INCLUDES Applecare
Services INCLUDES Cloud
Services INCLUDES Digital Content
Services I

{'query': "\nCan any trends be identified in Apple's Services segment revenue over the reported periods?\n",
 'result': "Yes, based on the provided knowledge triplets, some trends can be identified in Apple's Services segment revenue over the reported periods. The following trends can be observed:\n\n1. There is a gross margin decrease in the Services segment in the third quarter of 2022 and 2023. This suggests that the profitability of the Services segment decreased during these periods.\n2. The gross margin decrease in the Services segment is attributed to various factors such as higher services costs, foreign currencies, U.S. Dollar, improved leverage, and services gross margin percentage.\n3. The gross margin decrease in the Services segment in the first nine months of 2023 is also mentioned.\n4. The Services segment's gross margin increase is linked to higher services net sales.\n5. The Services segment's impact on net sales is mentioned, indicating that it plays a significant rol

In [9]:
question = """
Are there any notable changes in Apple's liquidity position or cash flows as reported in these 10-Qs?
"""

chain.invoke(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Apple
Full Context:
Apple withdrew from the case

> Finished chain.


{'query': "\nAre there any notable changes in Apple's liquidity position or cash flows as reported in these 10-Qs?\n",
 'result': "I don't know."}